In [1]:
from read_db_json import *

In [2]:
from sklearn.metrics import r2_score, mean_squared_error

In [3]:
# date = '2017-09-29'
date='2017-08-30'

In [4]:
ratings = large_joint_table(date=date)

with test (409, 12)
without test users (409, 12)


In [5]:
ratings.shape

(636, 30)

In [6]:
y_all = weighted_seg_score(ratings)

In [8]:
y_all.shape

(407, 1)

In [10]:
y_all.to_csv('ys/%s/y_csl_all.csv' % date)

# EXP5 reconstruction

In [8]:
SAMPLING_SEEDS = [502, 7081, 5128, 5561, 6711]

In [15]:
for frac in range(90, 0, -10):
    print('====frac:', frac)
    for sample_seed in SAMPLING_SEEDS:
        sampled_ratings = ratings.sample(frac=frac/100.0, random_state=sample_seed)
        sampled_y = weighted_seg_score(sampled_ratings)
        print(sample_seed, sampled_ratings.shape, sampled_y.shape)
#         sampled_y.to_csv('ys/%s/exp5_reconstruction/y_csl_frac#%d_sample_%d.csv' % (date, frac, sample_seed))

====frac: 90
502 (572, 30) (299, 1)
7081 (572, 30) (298, 1)
5128 (572, 30) (299, 1)
5561 (572, 30) (302, 1)
6711 (572, 30) (297, 1)
====frac: 80
502 (509, 30) (284, 1)
7081 (509, 30) (279, 1)
5128 (509, 30) (285, 1)
5561 (509, 30) (290, 1)
6711 (509, 30) (283, 1)
====frac: 70
502 (445, 30) (267, 1)
7081 (445, 30) (260, 1)
5128 (445, 30) (262, 1)
5561 (445, 30) (272, 1)
6711 (445, 30) (261, 1)
====frac: 60
502 (382, 30) (244, 1)
7081 (382, 30) (239, 1)
5128 (382, 30) (238, 1)
5561 (382, 30) (244, 1)
6711 (382, 30) (246, 1)
====frac: 50
502 (318, 30) (214, 1)
7081 (318, 30) (207, 1)
5128 (318, 30) (214, 1)
5561 (318, 30) (213, 1)
6711 (318, 30) (218, 1)
====frac: 40
502 (254, 30) (182, 1)
7081 (254, 30) (178, 1)
5128 (254, 30) (184, 1)
5561 (254, 30) (182, 1)
6711 (254, 30) (187, 1)
====frac: 30
502 (191, 30) (147, 1)
7081 (191, 30) (148, 1)
5128 (191, 30) (144, 1)
5561 (191, 30) (149, 1)
6711 (191, 30) (151, 1)
====frac: 20
502 (127, 30) (106, 1)
7081 (127, 30) (110, 1)
5128 (127, 30) (

# find sub type worth experimenting

In [7]:
def get_stat(filtered_ratings, col, label):
    if filtered_ratings.shape[0]==0:
        return {'question': col, 'answer':label, '#data points': 0}
    else:
        cmp_all = y_all.join(filtered_ratings, lsuffix='_all', how='inner')    
        r2 = r2_score(cmp_all.csl_all, cmp_all.csl)
        rmse = pd.np.sqrt(mean_squared_error(cmp_all.csl_all, cmp_all.csl))
        same_class_pcnt = (cmp_all.csl_all.round()==cmp_all.csl.round()).mean()
        return {
            'question': col, 'answer': label, '#data points':filtered_ratings.shape[0],
            'r2' : r2, 'rmse': rmse, 'same_class_percentage': same_class_pcnt,
        }

In [8]:
y_shape_with_filtered = []
for col, labels in SURVEY_DICT.items():
    question_col = ratings[col].fillna(0).astype(int)
    for i, label in enumerate(labels):  
        filtered_ratings = weighted_seg_score(ratings[question_col==i])
        stat = get_stat(filtered_ratings, col, label)
        y_shape_with_filtered.append(stat)

col = 'experienceLevel'
question_col = ratings[col].fillna('No Info')
for label in ['No Info', 'fearless', 'confident', 'interested', 'reluctant']:
    filtered_ratings = weighted_seg_score(ratings[question_col==label])
    stat = get_stat(filtered_ratings, col, label)
    y_shape_with_filtered.append(stat)
    

In [9]:
df_y_shape=pd.DataFrame(y_shape_with_filtered)

In [28]:
df_y_shape[['question', 'answer', '#data points', 'r2', 'rmse', 'same_class_percentage']].to_csv('filtered_y_shape.csv')

# get sub ys

In [6]:
sub_ys = {}

for col, labels in SURVEY_DICT.items():
    # consider these 4 types+exp lvl only
    if col not in ['bk_purpose', 'household_income', 'age', 'bk_type']:
        continue
    question_col = ratings[col].fillna(0).astype(int)
    for i, label in enumerate(labels):  
        # exclude no Info
        if i==0:
            continue
        filtered_ratings = weighted_seg_score(ratings[question_col==i])
        if filtered_ratings.shape[0]>30:
            sub_ys[(col, label)] = filtered_ratings

col = 'experienceLevel'
question_col = ratings[col].fillna('No Info')
for label in ['No Info', 'fearless', 'confident', 'interested', 'reluctant']:
    filtered_ratings = weighted_seg_score(ratings[question_col==label])
    if filtered_ratings.shape[0]>30:
        sub_ys[(col, label)] = filtered_ratings

In [7]:
sub_ys.keys()

dict_keys([('bk_purpose', 'Mainly utility biking – traveling from one location to another'), ('bk_purpose', 'Mainly recreational biking – biking for recreation, leisure, and health'), ('bk_purpose', '50% utility, 50% recreational'), ('age', '25-34'), ('age', '35-44'), ('household_income', 'Less than $24,999'), ('household_income', '$25,000 to $39,999'), ('household_income', '$55,000 to $69,999'), ('household_income', '$70,000 to $84,999'), ('household_income', '$85,000 to $99,999'), ('household_income', '$100,000 to $149,999'), ('household_income', '$150,000 or more'), ('bk_type', 'Road Bike'), ('bk_type', 'Hybrid Bike'), ('experienceLevel', 'fearless'), ('experienceLevel', 'confident')])

In [16]:
for key, df in sub_ys.items():
    fn = 'ys/y_csl_%s#%s.csv' % (key[0], key[1].split('–')[0])
    df.to_csv(fn, encoding='utf8')